# 15. 사람의 몸짓을 읽어보자

# 15-1. 들어가며

### Human Pose Estimation with Keypoint detection
전 세계적으로 인기를 끌고 있는 틱톡 애플리케이션을 보면 지금까지 우리가 만들어온 얼굴인식과 달리 전신이 등장한다.

오늘은 human pose estimation에 대한 개념과 이론에 대해 알아보자.

# 15-2. body language, 몸으로 하는 대화

human pose estimation은 크게 2D와 3D로 나누어진다.

2D는 2D 이미지에서 x,y 2차원 좌표들을 찾아내고, 3D는 2D 이미지에서 x,y,z 3차원 좌표들을 찾아내는 기술이다.   
하지만 2차원 이미지에서 3차원 이미지를 복원하는 일은 굉장히 어려운 일이다.
- [영상 Geometry #1 좌표계](https://darkpgmr.tistory.com/77)
- [영상 Geometry #7 Epipolar Geometry](https://darkpgmr.tistory.com/83?category=460965)

실세계 좌표계에서 사람의 발은 바닥에 있으면서 무릎은 머리 위로 갈 수 없듯이, 사람의 몸은 3D 환경에서 제약이 있다. 그래서 이런 제약 조건을 이용하면 어느 정도 문제를 해결할 수 있다.

3D pose estimation을 깊이 있게 다루기에 매우 오랜 시간이 걸리기 때문에, 오늘은 2D 영상 내에서 (x, y) pose(관절)의 위치를 찾는 방법을 다뤄보자.

# 15-3. Pose는 face landmark랑 비슷해요

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/02_8LQHgwE.png)

2D pose estimation을 다룬 적이 있다. face landmark와 매우 비슷하다.   
딥러닝이나 사람의 시각에서도 실제로 매우 비슷한 앱이다. 입력과 출력 개수만 다를 뿐 상당히 비슷하다.

하지만 난이도에서 차이가 난다. face landmark는 물리적으로 거의 고정되어 있는데(입이 얼굴보다 클 수 없음), human pose는 팔, 다리가 상대적으로 넓은 범위와 자유도를 갖는 것을 고려해야 한다.

자유도가 높다는 것은 데이터 분포를 특정하기 어렵다고 표현할 수 있다. 데이터 분포 학습하기 어렵다 -> 학습에 더 많은 데이터가 필요하고 더 복잡한 모델을 사용해야함을 의미한다.

따라서 상당히 많은 사전 작업이 요구되고 사용하려는 앱에 따라 접근 방법도 달라진다.   
가장 초기에 만나는 접근법은 두 가지로 나눠질 수 있다.

### 우리에게 맞는 방법은 뭘까?
![](https://d3s0tskafalll9.cloudfront.net/media/images/03_6C5RZR1.max-800x600.png)

첫 번째 방법은 Top-down 방법이다.   
- 모든 사람의 정확한 keypoint를 찾기 위해 object detection을 사용한다.
- crop한 이미지 내에서 keypoint를 찾아내는 방법으로 표현한다.
- detector가 선행되어야 하고 모든 사람마다 알고리즘을 적용해야 하기 때문에, 사람이 많이 등장할 때는 느리다는 단점이 있다.

두 번째 방법은 Bottom-up 방법이다.
- detector가 없고 keypoint를 먼저 검출한다. 예를들어 손목에 해당하는 모든 점들을 검출한다.
- 한 사람에 해당하는 keypoint를 clustering 한다.
- detector가 없기 때문에 다수의 사람이 영상에 등장하더라도 속도 저하가 크지 않다. 반면 top down 방식에 비해 keypoint 검출 범위가 넓어 성능이 떨어지는 단점이 있다.

얼마나 정확해야 하는지, 여러 사람이 등장하는지에 따라 필요한 알고리즘이 달라질 수 있다. 핸드폰 카메라로 찍는 인물들은 대체로 소수가 등장하기 때문에 top-down 방식을 이용해도 큰 속도 저하 없이 사용할 수 있을 것이라 생각한다.

Top-down 방법들에 대해 자세히 알아보자.

# 15-4. human keypoint detection(1)

### 자유도가 높은 사람의 동작
![](https://github.com/Team-Neighborhood/Kalman-Filter-Image/raw/master/result/KF_result.gif)

human pose estimation은 keypoint의 localization 문제를 푼다는 점에서 비슷하다.   
하지만 손목, 팔꿈치 등의 joint keypoint 정보는 얼굴의 keypoint보다 훨씬 다양한 위치와 변화를 보인다.

위 gif에서 볼 수 있듯이 손이 얼굴을 가리는 행위, 모든 keypoint가 영상에 담기지 않는 invisible, occlusions, clothing, lighting change가 face landmark에 비해 더 어려운 환경을 만든다.

딥러닝 기반 방법이 적용되기 전에는 다양한 사전 지식이 사용됐다.   
가장 기본이 되는 아이디어는 인체는 변형 가능 부분으로 나눠져 있고 각 부분끼리 연결성을 갖고 있다는 것이다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/05_9oDmNOY.max-800x600.png)

그림에서 보이는 것처럼 손은 팔, 팔은 몸과 연결되어 있다. 손이 다리 옆에 있을 확률이 팔 옆에 있을 확률보다 훨씬 작은 것이다. 이런 제약 조건을 그림에 보이는 스프링으로 표현했다.

3D 환경에서 생각하면 좋은 방법이다. 하지만 우리가 다루는 2D 데이터는 촬영 각도에 따라 충분히 팔이 다리 옆에서 관찰될 수 있다.

이 문제를 해결하기 위해 Deformable park models 방법에서는 각 part들의 complex joint relationship의 mixture model로 keypoint를 표현하는 방법을 이용했지만 성능은 기대에 미치지 못했다.
- [Articulated human detection with flexible mixtures-of-parts](https://www.cs.cmu.edu/~deva/papers/pose_pami.pdf) 관련 논문

deformable part model은 template들의 모음으로 구성되어 있다. 그리고 이는 global과 part 두 가지 종류로 나눠져 있다.

### DeepPose
딥러닝 이전의 전통적 pose estimation 모델은 분명한 한계가 있었다. deformable parts model 논문에서 언급했듯이 graphical tree model은 같은 이미지에 두 번 연산을 하는 등 연산 효율이 떨어지는 점과 그에 비해서도 부족한 성능이 문제점으로 인식되어 왔다.

AlexNet 이후, 다양한 분야에 CNN이 적용되면서 pose estimation 분야에도 CNN을 이용한 방법이 나타나기 시작했다. Toshev and Szegedy는 처음으로 딥러닝 기반 keypoint localization 모델을 제안했다.
- [DeepPose: Human Pose Estimation via Deep Neural Networks](https://arxiv.org/pdf/1312.4659.pdf)

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/06_ihhZoAI.png)

기존 기술로 풀기 어려웠던 동작의 다양성, invisible joint의 문제를 언급하며 딥러닝 기반 추론 방법이 해결책이 될 수 있다는 것을 증명했다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/07_GOoFtrb.png)

초기의 pose estimation 모델은 x, y 좌표를 직접적으로 예측하는 position regression 문제로 인식했다. human detection을 통한 crop된 사람 이미지를 이용해서 딥러닝 모델에 입력하고 (x,y) 좌표를 출력하도록 만든다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/08_iBnIDWo.png)

DeepPose는 매우 혁신적인 시도였지만 사실 성능이 압도적으로 높았던 것은 아니다.   
표에서 볼 수 있듯이 DeepPose가 전반적으로 높은 성능을 나타내고 있긴 하지만 기존 Tree based model인 Wang et al.의 방법에 비해 비약적으로 성능 향상이라 말하기 어렵다. DeepPose의 기여는 SOTA에 가까운 성능을 내면서 딥러닝을 적용한 첫 번째 사례라고 할 수 있다.

### Efficient Object Localization Using Convolutional Network
DeepPose는 딥러닝을 사용했는데 왜 성능이 비약적으로 상승하지 않았을까?

![](https://d3s0tskafalll9.cloudfront.net/media/images/09_NSfxHyY.max-800x600.png)

Tompson이 제안한 Efficient object localization 방법을 알아보자.   
이 논문에서는 제안했던 모델도 DeepPose에 비해 깊어졌지만, 가장 중요한 건 keypoint의 위치를 직접 예측하기보다 keypoint가 존재할 확률 분포를 학습하게 하자는 점이다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/10_SBHBawM.max-800x600.png)

human pose(keypoint)도 사람이 labeling을 할 수 밖에 없는데 사람이 항상 같은 위치의 점을 찍을 수 있을까?   
아까 위의 gif를 보면 kalman filter라고 적힌 동영상에 비해 orig measured는 점이 굉장히 떨리고 있다. 항상 같은 위치라고 생각하면서 keypoint를 선택하지만 사실 매 사진마다 수 픽셀씩 차이가 생기고 있다.

자연상태에서 일어나는 확률 분포는 가우시안 분포일 가능성이 크다. Tompson은 이런 점에 착안하여 label을 (x,y) 좌표에서 (x,y)를 중심으로 하는 heatmap으로 변환했다. 딥러닝 모델은 이 heatmap을 학습하게 되는 것이다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/11_bVXlDCF.png)

keypoint가 존재할 확률을 학습하게 된 이후로 성능이 비약적으로 향상되는 모습을 볼 수 있다.

Toskev가 제안한 DeepPose에 비해 2배가 넘는 수치를 볼 수 있다. 머리의 경우 0.9가 넘는 높은 성능을 갖게 됐다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/12_VPfjvOz.max-800x600.png)

MPII 데이터는 2014년에 나왔다. 기존 FLIC 데이터가 머리, 어깨, 팔꿈치, 손목 수준의 적은 keypoint를 갖고 있었지만 MPII는 몸의 각 관절 불위 16개의 keypoint를 갖는다. 기존 논문(Gkioxari, Sapp)들이 일부 데이터가 없는 이유이다. MPII는 실습에서 자세히 다뤄보자.

Tompson이 제안한 방법은 heatmap 학습뿐만이 아닌 모델에서도 개선을 이뤘다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-9-L-Casecaded.max-800x600.png)

사진을 살펴보면 coarse model과 fine model로 나누어지는데 coarse model에서 32x32 heatmap을 추출한 후 multi resolution 입력을 coarse heatmap 기준으로 crop한 후에 fine model에서 refinement를 수행한다.

그리고 coarse model과 fine model이 같은 모델로 weight를 공유한다. 목적이 같아 빠른 학습이 가능하고 메모리, 저장공간을 효율적으로 사용할 수 있다.

# 15-5. human keypoint detection(2)

### Convolutional Pose Machines
CVPR 2016에서 발표한 CPM은 completely differentiable한 multi-stage 구조를 제안했다.   
해당 방법들은 DeepPose부터 지속적으로 사용되어 왔었다. 하지만 crop 연산 등 비연속적인 미분 불가능한 stage 단위로 나눠져 있었기 때문에 학습 과정을 여러 번 반복하는 비효율적인 방법을 사용했다.
- [Convolutional Pose Machines](https://arxiv.org/pdf/1602.00134.pdf)

![](https://d3s0tskafalll9.cloudfront.net/media/images/13_wp3QD5J.max-800x600.png)

![](https://d3s0tskafalll9.cloudfront.net/media/images/14.max-800x600.png)

CPM은 end-to-end로 학습할 수 있는 모델을 제안한다.

Stage 1은 image feature를 계산하는 역할이고 stage 2는 keypoint를 예측하는 역할을 한다.   
g1, g2 모두 heatmap을 출력하게 만들어서 재사용이 가능한 부분은 weight sharing 할 수 있도록 세무 모델을 설계했다.

Stage >= 2에서 볼 수 있듯이 stage 2 이상부터는 반복적으로 사용할 수 있다. 보통 3개의 스테이지를 사용한다고 한다.   
stage 1 구조는 고정, stage 2부터는 해당 구조를 반복해서 추론한다. stage 2부터는 입력이 heatmap(image feature)이 되기 때문에 stage 단계를 거칠수록 keypoint가 refinement가 되는 효과를 볼 수 있다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/15.max-800x600.png)

CPM이 아주 좋은 방법은 아니다 Multi-stage 방법을 사용하기 때문에 end-to-end로 학습이 가능하더라도 그대로 학습하는 경우는 높은 성능을 달성하기 어렵다.   
따라서 stage 단위로 pretraining을 한 후에 다시 하나의 모델로 합쳐서 학습을 한다. 논문을 작성하기 위해 충분히 감내할 수 있지만 서비스 측면에서는 불편한 요소이다. 이러한 문제점들은 후에 제안되는 모델들이 적극적으로 개선하고 있다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/16.max-800x600.png)

CPM을 다루는 이유는 성능 때문이다. receptive field를 넓게 만드는 multi stage refinement 방벙비 성능 향상에 크게 기여한 것 같다.   
주황색 실선이 Tompson 알고리즘이다. CPM에서 제안한 검정색, 회색 실선이 detection rate에서 유의미한 차이를 보이고 있다. MPII의 PCKh@0.5에서 87.95%를 달성했다고 한다. 당시 2등보다 6.11%p 높은 성능을 보였다.

### Stacked Hourglass Network
ECCV16에서 DeepPose 이후 랜드마크라고 불릴만한 논문이 제안되었다. [Stacked Hourglass Networks for Human Pose Estimation](https://arxiv.org/pdf/1603.06937.pdf)이다.

__Hourglass__   
Stacked hourglass Network의 기본 구조는 모래시계와 같은 모양으로 만들어져 있다. Conv layer와 pooling으로 이미지(feature)를 인코딩하고 upsampling layer를 통해 feature map의 크기를 키우는 방향으로 decoding한다.   
feature map의 크기가 작아졌다가 커지는 구조여서 hourglass라고 표현한다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/17.max-800x600.png)

기존 방법들과 가장 큰 차이점은 feature map upsampling, residual connection이다.

pooling으로 image의 global feature를 찾고 upsampling으로 local feature를 고려하는 아이디어가 hourglass의 핵심 novelty라고 할 수 있따.   
hourglass의 모델 구조는 U-Net과 비슷하다. 간단한 구조를 여러 층으로 쌓아올려서 human pose estimation의 성능을 향상시켰다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/18.max-800x600.png)

MPII에서 처음으로 PCKh@0.5 기준 90%를 넘어서는 성과를 보이게 된다. 특유의 간단한 구조와 높은 성능으로 현재까지도 많이 사용되고 있는 구조이다.

### SimpleBaseline
앞서 소개한 연구들은 딥러닝 기반 2D human pose estimation이 어떻게 발전했는지 보여주고 있다. (x, y)를 직접 regression 하는 방법이 heatmap 기반으로 바뀌고 모델의 구조가 바뀌면서 encoder-decoder가 쌓여가는 형태가 완성됐다.

결과적으로 MPII에서 90%를 넘길 정도로 좋아졌지만 모델의 구조는 복잡해졌다.   
HPE의 연구를 하던 당시 microsoft 인턴 Haiping Wu는 다른 시각으로 보기 시작했다. "기술 자체가 많이 발전했는데 현재의 간단한 모델은 얼마나 성능이 좋을까?"   

[SimpleBase](https://arxiv.org/pdf/1804.06208.pdf)의 저자는 아주 간단한 encoder-decoder 구조를 설계한다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/19.max-800x600.png)

해당 구조로 73.7%의 AP를 COCO에서 달성한다. 직전 연도 2017년의 72.1% 결과를 뛰어넘는 수치이다. 해당 성과로 ECVV'18에 출판됐다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/20.max-800x600.png)

COCO에서 mAP 구치를 확인할 수 있다. 직전에 나온 hourglass와 비교해보면 다음과 같은 결과가 나온다고 한다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/21.max-800x600.png)

resnet-50만 사용하던 간단한 구조가 hourglass와 같은 SOTA를 이겼다는 것에 큰 파장을 가져온 논문이라 생각한다.

CPN은 Cascaded Pyramid Network이라는 모델이다. skip connection이 stage 사이에 연결되어 있다는 정도만 이해하고 넘어가자.   

다음 스텝에서 코드와 함께 살펴보자.

### Deep High-Resolution Network(HRNet)
[HRNet](https://arxiv.org/pdf/1902.09212.pdf)은 개발된 이후 현재까지 SOTA에 가까운 성능을 보일정도로 좋은 알고리즘이다. Simplebaseline의 1저자가 참여해 연구한 모델로 Simplebaseline과 같은 철학을 공유한다.

Stacked hourglass, Casecaded pyramid network 등은 multi-stage 구조로 이루어져 있어 학습과 추론 속도가 느리다는 큰 단점이 있다.(대신 하이퍼파라미터를 최적화 할 경우 1-stage 방법보다 성능이 좋다.)   
반면 Simplebaseline과 HRNet은 간단하는 추구하는 만큼 1-stage를 고수한다. 덕분에 구조도 간결해지고 사용하기도 쉽다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/22.max-800x600.png)

COCO dataset에서 한때 SOTA 성능을 자랑했다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/23.max-800x600.png)   
1-stage에서 어떻게 모델을 변화 시켰을까? 기존 알고리즘을 먼저 살펴보자.   
a는 Hourglass, b는 CPN(cascaded pyramid networks), c는 simplebaseline - tranposed conv, d는 simplebaseline - dilated conv를 나타냈다.

c, d의 simplebaseline은 다른 알고리즘에 비해 성능이 떨어지진 않지만 구조적으로 공통점과 차이점을 관찰할 수 있다.   
공통점으로는 high -> low resolution인 encoder와 low -> high인 decoder 구조로 이루어져있다.   
차이점은 encoder가 resnet50같은 backbone을 사용하면서 skip connection을 사용하면서 encoder가 무섭지만 simplebaseline은 skip connection이 없다.

그렇다면 왜 사용하지 않았을까?   
pooling을 할 때 소실되는 정보를 high level layer에서 사용하여 detail한 정보를 학습하기 위해 사용한다. 하지만 당연히 사용할 때 성능이 더 좋다.

HRNet의 저자도 위 질문에 대해 고민을 했다. high -> low -> high 구조에서 high resolution representation을 유지할 수 있는 모델을 어떻게 만들 수 있을까?   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/24.png)

고민의 결과 down sample layer를 만들고 작아진 layer feature 정보를 다시 up sampling을 통해 원본 해상도 크기에 적용하는 모델을 제안했다.   
다소 복잡해 보이지만 1-stage로 동작하기 때문에 전체 flow를 보면 엄청 간단하다.   
앞에서 다뤘던 CPM이나 Hourglass는 중간 단계의 heatmap supervision이 학습 과정에 꼭 필요했는데 HRNet은 필요가 없다.

구현도 Simplebaseline의 backbone인 ResNet을 HRNet으로 교체만 해주면 되기 떄문에 사용하기도 굉장히 편리하다.
HRNet 또한 이전 알고리즘들과 마찬가지로 heatmap을 regression 하는 방식으로 학습하고 MSE loss를 이용한다.(simplebaseline과 거의 유사하다)

![](https://d3s0tskafalll9.cloudfront.net/media/images/25.max-800x600.png)

결과를 보면 HRNet이 4%에 가까운 비약적인 성능 향상을 이뤄냈다. 비교적 학습이 간단하면서 성능까지 좋은 모델이라 현재도 많이 사용되고 있다.   
원저자의 pytorch 코드가 매우 깔끔하게 구현되어 있어 재생산성이 높아 사용하기 좋다.   
- [https://github.com/leoxiaobin/deep-high-resolution-net.pytorch](https://github.com/leoxiaobin/deep-high-resolution-net.pytorch)

# 15-6. 코드로 이해하는 pose estimantion

오늘 본 6개의 모델 중 가장 간단한 모델인 SimpleBaseline 모델 부분만 정확하게 구현하여 이해해보자.

### SimpleBaseline 구조
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/26.png)

encoder는 conv layers, decoder는 deconv module + upsampling으로 이루어져 있다.   

해당 conv layer가 어떻게 이루어져 있는지, devonv module은 어떻게 되어 있는지, deconv module 이 그림처럼 3개일지는 논문을 읽어봐야 알 수 있다.   
[Simple Baselines for Human Pose Estimation and Tracking](https://arxiv.org/pdf/1804.06208.pdf)

backbone으로 resnet을 사용한다.   
deconv module은 deconv-Batch-relu가 3개로 이루어져 있다. deconv는 256 filter, 4x4 kernel, stride 2로 2배씩 feature map이 커진다.   
마지막 출력 레이어는 $k$개의 1x1 conv layer로 heatmap을 generate predicted한다.

### PyTorch code 읽어보기
[https://github.com/Microsoft/human-pose-estimation.pytorch](https://github.com/Microsoft/human-pose-estimation.pytorch)

논문도 모든 디테일을 설명해주지 않는다. 인공지능 분야의 최대 장점은 저자의 공식 코드가 제공된다는 점이다.   
simplebaseline의 저자도 논문에서 코드 repo의 위치를 언급했다.

18년 이후 pytorch의 성장으로 절반 이상의 오픈소스가 파이토치로 공개되고 있다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/27.max-800x600.png)   
주요 컨퍼런스의 pytorch, tensorflow의 성장률이다. 연구 분야에서 pytorch가 압도적으로 많이 사용되고 있다.

[microsoft/human-pose-estimation.pytorch](https://github.com/microsoft/human-pose-estimation.pytorch/blob/master/lib/models/pose_resnet.py) 모델 부분을 읽어보자.

`nn` 표현이 많이 등장한다. `torch.nn`으로 `keras.layers`와 같이 딥러닝 모델 구성에 필요한 도구들이 정의되어 있다.   
29번째 줄 BasicBlock 클래스가 있다. keras.models로 model을 선언하는 것과 비슷하다.   

pytorch model에서는 사용된 layer를 forward 함수를 통해 computational graph를 그려준다.   

42번째 줄 forward 함수는 residual block을 사용했다.   

157번째 줄을 보면 메인 model에 4개의 residual block을 이용한다. 이는 resnet과 동일하다.

```
def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.deconv_layers(x)
        x = self.final_layer(x)

        return x
```

forward 함수를 봤을 때 resnet을 통과한 후 `deconv_layers`와 `final_layer`를 차례로 통과한다.

219번째 줄의 deconv layer를 보면   
```
        layers.append(
                nn.ConvTranspose2d(
                    in_channels=self.inplanes,
                    out_channels=planes,
                    kernel_size=kernel,
                    stride=2,
                    padding=padding,
                    output_padding=output_padding,
                    bias=self.deconv_with_bias))
            layers.append(nn.BatchNorm2d(planes, momentum=BN_MOMENTUM))
            layers.append(nn.ReLU(inplace=True))
```
transpose conv와 bn, relu로 이루어져 있는 것을 확인할 수 있다.   

세세한 파라미터는 EXTRA가 자주 등장하는걸 봤을 때, 어떤 configuration 파일이 있을 것으로 짐작해볼 수 있다. repo에서 검색해보면 파라미터 관련 정보를 담고 있는 파일을 찾을 수 있다.   
[https://github.com/microsoft/human-pose-estimation.pytorch/blob/master/experiments/coco/resnet50/256x192_d256x3_adam_lr1e-3.yaml#L23](https://github.com/microsoft/human-pose-estimation.pytorch/blob/master/experiments/coco/resnet50/256x192_d256x3_adam_lr1e-3.yaml#L23)

```
NUM_DECONV_LAYERS: 3
    NUM_DECONV_FILTERS:
    - 256
    - 256
    - 256
    NUM_DECONV_KERNELS:
    - 4
    - 4
    - 4
```
deconv layer의 파라미터가 상세하게 적혀있다.

이렇게 simplebaseline의 모델을 상세하게 파악할 수 있었다.

192x256의 이미지가 입력될때 renset 출력이 6x8이 나오고 3개의 deconv layer를 통과해 48x64가 출력된다.

### simplebaseline - tf2
파악한 지식을 토대로 tensorflow simplebaseline 모델을 만들어보자.

In [1]:
import os

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet')

94781440/94765736 [==============================] - 1s 0us/step


In [2]:
upconv1 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn1 = tf.keras.layers.BatchNormalization()
relu1 = tf.keras.layers.ReLU()
upconv2 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn2 = tf.keras.layers.BatchNormalization()
relu2 = tf.keras.layers.ReLU()
upconv3 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn3 = tf.keras.layers.BatchNormalization()
relu3 = tf.keras.layers.ReLU()

deconv module 중복을 제거해보자.

In [3]:
def _make_deconv_layer(num_deconv_layers):
    seq_model = tf.keras.models.Sequential()
    for i in range(num_deconv_layers):
        seq_model.add(tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same'))
        seq_model.add(tf.keras.layers.BatchNormalization())
        seq_model.add(tf.keras.layers.ReLU())
    return seq_model

upconv = _make_deconv_layer(3)

In [4]:
final_layer = tf.keras.layers.Conv2D(17, kernel_size=(1,1), padding='same')

In [5]:
inputs = keras.Input(shape=(256, 192, 3))
x = resnet(inputs)
x = upconv(x)
out = final_layer(x)
model = keras.Model(inputs, out)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 192, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 sequential (Sequential)     (None, 64, 48, 256)       10489600  
                                                                 
 conv2d (Conv2D)             (None, 64, 48, 17)        4369      
                                                                 
Total params: 34,081,681
Trainable params: 34,027,025
Non-trainable params: 54,656
_________________________________________________________________


가상의 이미지를 넣어 출력이 잘 되는지 확인해보자.

In [6]:
np_input = np.zeros((1,256,192,3), dtype=np.float32)
tf_input = tf.convert_to_tensor(np_input, dtype=tf.float32)
print('input shape')
print (tf_input.shape)
print('\n')

tf_output = model(tf_input)
print('output shape')
print (tf_output.shape)
print (tf_output[0,:10,:10,:10])

input shape
(1, 256, 192, 3)


output shape
(1, 64, 48, 17)
tf.Tensor(
[[[ 1.12301810e-03  7.55356438e-03 -5.85100148e-03 -3.13820178e-03
   -7.37525849e-03 -2.96494062e-03  2.81174690e-03  6.87631546e-04
   -3.69152956e-04 -3.69123212e-04]
  [ 1.28022570e-03  1.68429445e-02 -1.87815186e-02  6.31378731e-03
    4.82132472e-03  6.44772779e-03 -1.59344543e-02  1.15692290e-02
   -4.74173529e-03 -8.99400096e-03]
  [-4.38117702e-03  2.58055832e-02 -7.25149410e-03  8.48388113e-03
   -3.30337323e-03 -1.30864577e-02  8.41084868e-03  1.08793154e-02
    1.81287667e-03  8.86158179e-03]
  [ 1.63132045e-02  9.74918995e-03 -1.93835236e-02  6.73894165e-03
   -1.78277753e-02  1.76498871e-02 -9.43202991e-03 -1.07784765e-02
   -2.23322995e-02  1.56888962e-02]
  [ 5.59044478e-04  5.55235185e-02 -2.42329743e-02  5.49104484e-03
   -1.16721243e-02 -5.55089163e-03  2.89102987e-04  5.92886051e-03
   -6.18943246e-03 -8.07856675e-03]
  [ 6.78506587e-03  1.85557567e-02 -1.32298907e-02  4.15522978e-03
   -2.043795